### Assumptions:
1. Collection is already opened. A function does not need to open a collection.

In [ ]:
import pymongo
import datetime

In [ ]:
from pymongo import MongoClient
client = MongoClient()

In [ ]:
db = client['stud28']

In [ ]:
print db.collection_names(include_system_collections=False)

In [ ]:
# creating new collection
collection = db.test

In [ ]:
# clean up the collection before start working
db.test.drop()
# db.restaurants.drop_index("grades.score")

In [ ]:
company = {'company_name':'TAU', 'company_description':'University'}
add_company(company)

In [ ]:
def add_company(company):
    collection = db.test
    #print collection.find_one({"company_name":company['company_name']})
    # validation: only insert if the compant does not exist
    if collection.find_one({"company_name":company['company_name']}) == None:
        # insertion
        company['num_of_positions'] = 0
        # add jobs_list attribute
        company['jobs_list'] = []
        collection.insert_one(company)
        print company['company_name'],"was successfully added"
    else:
        print"Company is already exists in the system"

In [ ]:
def add_job(job, company_name):
    collection = db.test
    # validation: only insert if there is an associated company
    company_doc = collection.find_one({"company_name":company_name})
    if company_doc == None:
        print "Company does not exist in the system"
    else:
    # insertion
        # add candidates_list attribute
        job['candidates_list'] = []
        # assign auto increment job id
        job_id = company_doc['num_of_positions']
        job["_id"] = str(job_id +1)
        # update the number of positions for this company
        collection.update_many({"company_name":company_name},{"$set":{"num_of_positions":job_id+1}})
        # update the jobs list for the company
        collection.update_many({"company_name":company_name},{"$addToSet":{"jobs_list":job}})

In [ ]:
job ={'job_name':'bi developer', 'location': 'Tel Aviv',
      'skills':['python','big data','mongodb'],'status':'open','publish_date':'01-02-2019'}
add_job(job, 'TAU')

In [1]:
print collection.find_one({"company_name":'TAU'})

NameError: name 'collection' is not defined

In [ ]:
def application(candidate, application_time, job_id):
    # from job_id to company?
    company_name = 'TAU'
    # validation: check that this email was not applied for this job
    candidates_list = collection.find({"company_name":company_name, "jobs_list._id":job_id})
    existing_emails = []
    for i in candidates_list['jobs_list']:
        print"sdfsdf", i
        existing_emails.append(i['email'])
    if candidate['email'] in existing_emails:
        print "The inserted email has already applied for this position"
    else:
    #insert
        #add the application time to the candidate
        candidate["application_time"] = application_time
        collection.update_many({"company_name":company_name, "jobs_list._id":job_id},{"$push":{"jobs_list.$.candidates_list":candidate}})

#     collection.update_many({"company_name":company_name, "jobs_list._id":job_id, "jobs_list.$.candidates_list.email":{"$nin": [candidate['email']]}},{"$push":{"jobs_list.$.candidates_list":candidate}})      

In [ ]:
application({'candidate_name':'laura', 'email':'laura@gmail.com',
'linkedin':'https://www.linkedin.com/in/laura/', 'skills':['python','sql']},
'01-02-2019 15:00:00', '1')

In [ ]:
print collection.find_one({"company_name":'TAU'})

In [ ]:
print collection.find_one({"company_name":"TAU", "jobs_list._id":'1',
                           "jobs_list.1.candidates_list.email":{"$ne": 'laura@gmail.com'}})

In [ ]:
print collection.find_one({"company_name":"TAU","jobs_list._id":'2', "jobs_list.candidates_list.email":{"$ne":"laura@gmail.com"}})